# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Обидин Егор Владимирович`

Группа: `DS-32`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [1]:
import folium
from folium import plugins
import geopandas as gpd
import pandas as pd
import shapely
from haversine import haversine

In [2]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
afghanistan_poligon = world[world['name'] == 'Afghanistan']['geometry'].values[0]
center_poligon_afganistan = afghanistan_poligon.centroid.coords[0][::-1]

In [3]:
def in_poligon_range(lat, lon):
    """Определение находится ли точка в полигоне, и определяет расстоние до центра полигона в метрах"""
    point_in_poligon = shapely.geometry.Point(lon, lat).within(afghanistan_poligon)
    range_to_point = haversine((lat, lon), center_poligon_afganistan)
    return point_in_poligon, range_to_point #* 1000

In [4]:
import random
def find_point_with_random_model(format_lat_lon=True):
    '''Поиск точки случайным образом'''
    find = False
    path = []
    count = 0
    while find == False:
        lat = random.uniform(-90, 90)
        lon = random.uniform(-180,180)
        find, dist = in_poligon_range(lat, lon)
        count += 1
        if format_lat_lon:
            path.append([lat, lon])
        else:
            path.append([lon, lat])
    if format_lat_lon:
        print("Точка (",lat,";",lon,")", "найдена за",count,"итераций")
    else:
        print("Точка (",lon,";",lat,")", "найдена за",count,"итераций")
    return path

In [5]:
import datetime
def show_path(path, real_time=False):
    map_ = folium.Map(location=list(center_poligon_afganistan), zoom_start = 2)
    if real_time:
        base_date = datetime.datetime.today()
        features = []
        for i, point in enumerate(path[:-1]):
            base_date += datetime.timedelta(hours=1)
            features.append(
                {
                    'type': 'Feature',
                    'geometry': 
                    {
                        'type': 'Point',
                        'coordinates': point[::-1],
                    },
                    'properties': 
                    {
                        'times': [base_date.strftime("%Y-%m-%d %H:%M:%S")],
#                         'color': 'green',
#                         'weight': 3,
                        'icon':'circle',
                    }
                }
            )
        base_date += datetime.timedelta(hours=1)
        features.append(
                {
                    'type': 'Feature',
                    'geometry': 
                    {
                        'type': 'Point',
                        'coordinates': path[-1][::-1],
                    },
                    'properties': 
                    {
                        'times': [base_date.strftime("%Y-%m-%d %H:%M:%S")],
                        'popup': "Afganistan"
                    }
                }
            )
        folium.plugins.TimestampedGeoJson(
            {
                'type': 'FeatureCollection',
                'features': features,
            }, period='PT1H', auto_play=False, loop=False).add_to(map_)

    else:
        for point in path[:-1]:
            folium.CircleMarker(location=point, 
                                radius = 5, 
                                fill_color="green", 
                                color="gray", 
                                fill_opacity = 0.9).add_to(map_)
        folium.CircleMarker(location=path[-1], 
                            radius = 5, 
                            popup="Afganistan",
                            fill_color="red", 
                            color="gray", 
                            fill_opacity = 0.9).add_to(map_)

    return map_

In [6]:
path_random = find_point_with_random_model()

Точка ( 35.51672111581043 ; 71.11148761226448 ) найдена за 1029 итераций


In [7]:
show_path(path_random, real_time=False)

In [8]:
import numpy as np

def find_borders(a, b, c, d, index, lat, lon,):
    """Пересчет координат границ сетки"""
    if index == 0:
        b = lon
        d = lat
        return a, b, c, d
    if index == 1:
        b = lon
        c = lat
        return a, b, c, d
    if index == 2:
        a = lon
        c = lat
        return a, b, c, d
    if index == 3:
        a = lon
        d = lat
        return a, b, c, d
    a += (lon - a) / 2
    b = (b - lon) / 2 + lon
    c += (lat - c) / 2
    d = (d - lat) / 2 + lat
    return a, b, c, d

def find_lat_lon(lat_begin, lon_begin):
    print('Поиск координат')
    
    a = -180
    b = 180
    c = -90
    d = 90
    
    lat = lat_begin
    lon = lon_begin
            
    print('Параметры сетки')
    print('a =', a ,'b =', b , 'c =', c ,'d =', d)
    print('Рассматриваемая точка')
    print('Широта =', lat ,'Долгота =', lon)
    print('__________________________________________________________')

    point_in_poligon , range_to_point = in_poligon_range(lat, lon)
    print('Точка попала в полигон? = ',point_in_poligon, 'Дальность = ', range_to_point)     
    
    new_lon = np.zeros(4)
    new_lat = np.zeros(4)
    
    path = [[lat, lon]]
    j  = 0
    while point_in_poligon == False:
        if j > 100:
            break
        index = -1
        
        new_lon[0] = (lon - a) / 2 + a
        new_lon[1] = (lon - a) / 2 + a
        new_lon[2] = (b - lon) / 2 + lon
        new_lon[3] = (b - lon) / 2 + lon
        
        new_lat[0] = (lat - c) / 2 + c
        new_lat[1] = (d - lat) / 2 + lat
        new_lat[2] = (d - lat) / 2 + lat
        new_lat[3] = (lat - c) / 2 + c
        
        for i in range(4):
            j+=1
            point_in_poligon , dist = in_poligon_range(new_lat[i], new_lon[i])
            path.append([new_lat[i], new_lon[i]])
            if point_in_poligon == False:
                if dist < range_to_point:
                    range_to_point = dist
                    index = i
            else:
                print('Расчёт окончен')
                print('Широта =', new_lat[i] ,'Долгота =', new_lon[i], 'Дальность =', dist)
#                 path.append([new_lat[i], new_lon[i]])
                return path
        
        a, b, c, d = find_borders(a, b, c, d, index, lat, lon)
#         path.append([new_lat[i], new_lon[i]])
        if index > -1:
            lat = new_lat[index]
            lon = new_lon[index]
            
        print('Широта =', lat ,'Долгота =', lon, 'Дальность =', range_to_point, "index =", index)       
        
        print('Уменьшение сетки')
        print('a = ', a ,'b = ', b , 'c = ', c ,'d = ', d)
        print('__________________________________________________________')
    
    print('Расчёт окончен')
    print('Широта =', lat ,'Долгота =', lon, 'Дальность =', range_to_point)
    return path

In [9]:
path_to_Afganistan = find_lat_lon(random.uniform(-90, 90), random.uniform(-180, 180))

Поиск координат
Параметры сетки
a = -180 b = 180 c = -90 d = 90
Рассматриваемая точка
Широта = -8.468655235608693 Долгота = -177.1743183347043
__________________________________________________________
Точка попала в полигон? =  False Дальность =  12992.792512830683
Широта = 40.76567238219565 Долгота = 1.412840832647845 Дальность = 5642.143911623241 index = 2
Уменьшение сетки
a =  -177.1743183347043 b =  180 c =  -8.468655235608693 d =  90
__________________________________________________________
Широта = 16.14850857329348 Долгота = 90.70642041632392 Дальность = 3151.8141675414827 index = 3
Уменьшение сетки
a =  1.412840832647845 b =  180 c =  -8.468655235608693 d =  40.76567238219565
__________________________________________________________
Широта = 28.457090477744565 Долгота = 46.059630624485884 Дальность = 1994.112012793279 index = 1
Уменьшение сетки
a =  1.412840832647845 b =  90.70642041632392 c =  16.14850857329348 d =  40.76567238219565
________________________________________

In [10]:
show_path(path_to_Afganistan, real_time=True)

In [11]:
coord_Afganistan = path_to_Afganistan[-1].copy()
coord_Afganistan

[34.61138142997011, 68.3830255204049]

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``